[View in Colaboratory](https://colab.research.google.com/github/dhanaji/Machine-Learning-with-Python/blob/master/Text_20Classification_20Keras.ipynb)

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

In [2]:
import itertools
import os

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.preprocessing import LabelBinarizer, LabelEncoder
from sklearn.metrics import confusion_matrix

from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.preprocessing import text, sequence
from keras import utils

# This code was tested with TensorFlow v1.4
print("You have TensorFlow version", tf.__version__)

You have TensorFlow version 1.8.0


Using TensorFlow backend.


In [3]:
import urllib
urllib.urlretrieve('https://data.consumerfinance.gov/api/views/s6ew-h6mp/rows.csv?accessType=DOWNLOAD','Consumer_Complaints.csv')

('Consumer_Complaints.csv', <httplib.HTTPMessage instance at 0x7f795db2a248>)

In [5]:
df = pd.read_csv('Consumer_Complaints.csv', encoding='latin-1')
df.head()

,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID
0,03/12/2014,Mortgage,Other mortgage,"Loan modification,collection,foreclosure",NaN,NaN,NaN,M&T BANK CORPORATION,MI,48382,NaN,NaN,Referral,03/17/2014,Closed with explanation,Yes,No,759217
1,10/01/2016,Credit reporting,NaN,Incorrect information on credit report,Account status,I have outdated information on my credit repor...,Company has responded to the consumer and the ...,"TRANSUNION INTERMEDIATE HOLDINGS, INC.",AL,352XX,NaN,Consent provided,Web,10/05/2016,Closed with explanation,Yes,No,2141773
2,10/17/2016,Consumer Loan,Vehicle loan,Managing the loan or lease,NaN,I purchased a new car on XXXX XXXX. The car de...,NaN,"CITIZENS FINANCIAL GROUP, INC.",PA,177XX,Older American,Consent provided,Web,10/20/2016,Closed with explanation,Yes,No,2163100
3,06/08/2014,Credit card,NaN,Bankruptcy,NaN,NaN,NaN,AMERICAN EXPRESS COMPANY,ID,83854,Older American,NaN,Web,06/10/2014,Closed with explanation,Yes,Yes,885638
4,09/13/2014,Debt collection,Credit card,Communication tactics,Frequent or repeated calls,NaN,NaN,"CITIBANK, N.A.",VA,23233,NaN,NaN,Web,09/13/2014,Closed with explanation,Yes,Yes,1027760


In [6]:
col = ['Consumer complaint narrative', 'Product']
df = df[col]
df = df[pd.notnull(df['Consumer complaint narrative'])]
df.head()

,Consumer complaint narrative,Product
1,I have outdated information on my credit repor...,Credit reporting
2,I purchased a new car on XXXX XXXX. The car de...,Consumer Loan
7,An account on my credit report has a mistaken ...,Credit reporting
12,This company refuses to provide me verificatio...,Debt collection
16,This complaint is in regards to Square Two Fin...,Debt collection


In [7]:
df.isnull().sum()

Consumer complaint narrative    0
Product                         0
dtype: int64

In [8]:
df['Product'].value_counts()

Debt collection                                                                 65791
Credit reporting, credit repair services, or other personal consumer reports    53057
Mortgage                                                                        44793
Credit reporting                                                                31593
Credit card                                                                     18842
Student loan                                                                    17167
Bank account or service                                                         14887
Credit card or prepaid card                                                     11736
Consumer Loan                                                                    9474
Checking or savings account                                                      7071
Money transfer, virtual currency, or money service                               3393
Vehicle loan or lease                                 

In [9]:
# Split data into train and test
train_size = int(len(df) * .8)
print ("Train size: %d" % train_size)
print ("Test size: %d" % (len(df) - train_size))

Train size: 230581
Test size: 57646


In [10]:
train_narrative = df['Consumer complaint narrative'][:train_size]
train_product = df['Product'][:train_size]

test_narrative = df['Consumer complaint narrative'][train_size:]
test_product = df['Product'][train_size:]

In [11]:
max_words = 1000
tokenize = text.Tokenizer(num_words=max_words, char_level=False)

In [12]:
tokenize.fit_on_texts(train_narrative) # only fit on train
x_train = tokenize.texts_to_matrix(train_narrative)
x_test = tokenize.texts_to_matrix(test_narrative)

In [13]:
# Use sklearn utility to convert label strings to numbered index
encoder = LabelEncoder()
encoder.fit(train_product)
y_train = encoder.transform(train_product)
y_test = encoder.transform(test_product)

In [14]:
# Converts the labels to a one-hot representation
num_classes = np.max(y_train) + 1
y_train = utils.to_categorical(y_train, num_classes)
y_test = utils.to_categorical(y_test, num_classes)

In [15]:
# Inspect the dimenstions of our training and test data (this is helpful to debug)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)
print('y_train shape:', y_train.shape)
print('y_test shape:', y_test.shape)

x_train shape: (230581, 1000)
x_test shape: (57646, 1000)
y_train shape: (230581, 18)
y_test shape: (57646, 18)


In [16]:
batch_size = 32
epochs = 5

In [17]:
# Build the model
model = Sequential()
model.add(Dense(512, input_shape=(max_words,)))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [18]:
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1)

Train on 207522 samples, validate on 23059 samples
Epoch 1/5
161248/207522 [======================>.......] - ETA: 10s - loss: 0.9025 - acc: 0.7139

207522/207522 [==============================] - 49s 236us/step - loss: 0.8832 - acc: 0.7181 - val_loss: 1.1725 - val_acc: 0.5306
Epoch 2/5
 86176/207522 [===========>..................] - ETA: 27s - loss: 0.7538 - acc: 0.7517

207522/207522 [==============================] - 48s 232us/step - loss: 0.7584 - acc: 0.7520 - val_loss: 1.2289 - val_acc: 0.5324
Epoch 3/5
 58080/207522 [=======>......................] - ETA: 33s - loss: 0.6791 - acc: 0.7751

207522/207522 [==============================] - 49s 234us/step - loss: 0.7037 - acc: 0.7691 - val_loss: 1.2679 - val_acc: 0.5286
Epoch 4/5
 46560/207522 [=====>........................] - ETA: 36s - loss: 0.6265 - acc: 0.7917

207522/207522 [==============================] - 48s 234us/step - loss: 0.6557 - acc: 0.7842 - val_loss: 1.2291 - val_acc: 0.5548
Epoch 5/5
 42272/207522 [=====>........................] - ETA: 37s - loss: 0.5919 - acc: 0.8038

207522/207522 [==============================] - 48s 233us/step - loss: 0.6153 - acc: 0.7974 - val_loss: 1.3626 - val_acc: 0.5342


In [19]:
# Evaluate the accuracy of our trained model
score = model.evaluate(x_test, y_test,
                       batch_size=batch_size, verbose=1)
print('Test score:', score[0])
print('Test accuracy:', score[1])

57646/57646 [==============================] - 4s 70us/step
Test score: 1.4960747232060105
Test accuracy: 0.499080595353738


In [20]:
# Here's how to generate a prediction on individual examples
text_labels = encoder.classes_ 

for i in range(10):
    prediction = model.predict(np.array([x_test[i]]))
    predicted_label = text_labels[np.argmax(prediction)]
    print(test_narrative.iloc[i][:50], "...")
    print('Actual label:' + test_product.iloc[i])
    print("Predicted label: " + predicted_label + "\n")

To Chase Bank My wallet might 've gotten stolen or ...
Actual label:Checking or savings account
Predicted label: Bank account or service

XXXX XXXX is not my name, I never lived at some ad ...
Actual label:Credit reporting, credit repair services, or other personal consumer reports
Predicted label: Credit reporting

Equifax is reporting an account with XXXX XXXX XXX ...
Actual label:Credit reporting, credit repair services, or other personal consumer reports
Predicted label: Credit reporting

I 've disputed this account before, and still have ...
Actual label:Credit reporting, credit repair services, or other personal consumer reports
Predicted label: Credit reporting

On XX/XX/XXXX I received a phone call stating that ...
Actual label:Debt collection
Predicted label: Debt collection

I noticed inquiries on my Equifax credit report th ...
Actual label:Credit reporting, credit repair services, or other personal consumer reports
Predicted label: Credit reporting

When attempting to submi